In [1]:
#base_packs
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import statsmodels.api as sm
import sklearn.manifold as skm
import sklearn.cluster as skc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from statsmodels.stats.diagnostic import het_white
import ruptures as rpt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import statsmodels.api as sm
import sklearn.preprocessing as skp
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
from sklearn.cluster import KMeans
import gc
import warnings
from scipy.optimize import OptimizeWarning
import time
import numpy as np
import random
from sklearn.preprocessing import RobustScaler, QuantileTransformer,MinMaxScaler
from sklearn.metrics import calinski_harabasz_score

In [2]:
dt = 'V:/Departamentos/DVM/Transformação Digital/Projetos/064 - Otimização de Processo com IA no VOD (Ciência de Dados)/VOD_JUL_2024.xlsx'
dt = pd.read_excel(dt)
dt = pd.DataFrame(dt)
dt['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
dt['Tempo de depressão de vácuo(min)']= pd.to_numeric(dt['Tempo de depressão de vácuo(min)'] , errors = 'coerce')

dt['Teor de C após a descarburação(%)'].replace(',' , '.', inplace = True)
dt['Teor de C após a descarburação(%)']= pd.to_numeric(dt['Teor de C após a descarburação(%)'] , errors = 'coerce')
dt.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação'], inplace=True)
dt.replace('Boa', 1 , inplace = True)
dt.replace('Ruim', 0 , inplace = True)
dt = pd.get_dummies(dt)
for i in dt.columns:
    median_value = dt[i].median()  
    dt[i].fillna(median_value, inplace=True)
dt = dt.replace({False: 0, True: 1})


C:\Users\S409478\AppData\Local\Temp\ipykernel_15888\1447092083.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dt['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
C:\Users\S409478\AppData\Local\Temp\ipykernel_15888\1447092083.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [4]:
teste = 'C:/Users/S409478/Documents/vm_report_vo1725621775091.xls'
teste = pd.read_excel(teste)
teste = pd.DataFrame(teste)
# Extract the first row and set as new header
new_header = teste.iloc[0]
teste = teste[1:]  # Drop the first row
teste.columns = new_header  # Set new header

# Reset index
teste.reset_index(drop=True, inplace=True)

# Function to format numbers
def insert_decimal(x):
    x_str = str(x)
    
    # Handle cases where there is already a decimal point
    if '.' in x_str:
        integer_part, decimal_part = x_str.split('.')
        # If the integer part is shorter than 2 digits, just return as is
        if len(integer_part) <= 3:
            return x_str
        # Otherwise, insert decimal point after the second-to-last digit
        return integer_part[:-3] + '.' + integer_part[-3:] + (f".{decimal_part}" if decimal_part else "")
    
    # Handle integer values
    if len(x_str) > 3:
        return x_str[:-3] + '.' + x_str[-3:]
    return x_str

# Apply the function to each element in the DataFrame column
teste['Tempo abaixo de 10 mbar(min)'] = teste['Tempo abaixo de 10 mbar(min)'].apply(insert_decimal)
#teste['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
#teste['Tempo de depressão de vácuo(min)']= pd.to_numeric(dt['Tempo de depressão de vácuo(min)'] , errors = 'coerce')

#teste['Teor de C após a descarburação(%)'].replace(',' , '.', inplace = True)
#teste['Teor de C após a descarburação(%)']= pd.to_numeric(dt['Teor de C após a descarburação(%)'] , errors = 'coerce')
#teste.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação'], inplace=True)
#teste.replace('Boa', 1 , inplace = True)
#teste.replace('Ruim', 0 , inplace = True)
#teste = pd.get_dummies(teste)
#for i in teste.columns:
#    median_value = teste[i].median()  
#    teste[i].fillna(median_value, inplace=True)
#teste = teste.replace({False: 0, True: 1})


In [5]:
teste

,CORRIDA,AÇO,DATA,Teor de C antes do VOD(%),Teor de Cr antes do VOD(%),Teor de Si antes do VOD(%),Teor de Mn antes do VOD(%),Teor de Ni antes do VOD(%),Teor de Mo antes do VOD(%),Temperatura do aço antes do VOD(°C),...,Tempo de depressão de vácuo(min),Tempo total de descarburação(min),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Pressão final de vácuo na descarburação(mbar),Agitação média do TruStir durante a descarburação,%CO no final da descarburação,Volume de O2 soprado(m3),Teor de C após a descarburação(%),Temperatura do aço após a descarburação(°C)
0,820720,VSM13C,"9/4/24, 5:53 PM",0.45,12.62,0.2,0.19,5.89,2.1,1534,...,3017,224167,19.383,6.35,1.68,266773,11726,178,0.005,1686
1,820723,VSM13C,"9/5/24, 11:15 AM",0.69,12.73,0.19,0.22,5.93,2.09,1547,...,4933,236667,18.717,6.4,1.8,301497,17021,198,0.007,1692
2,820724,VSM13C,"9/5/24, 2:42 PM",0.6,12.5,0.2,0.19,5.88,2.1,1579,...,3.5,21,17.483,7,1.76,288614,10294,183,0.007,1713
3,820725,VSM13C,"9/5/24, 10:09 PM",0.75,12.79,0.2,0.17,6.04,2.14,1567,...,6767,24.15,17.35,5.67,1.96,301349,18916,208,0.005,1683


In [6]:
##normalization
scaler = skp.RobustScaler()
dt_k = scaler.fit_transform(dt)
kmeans = KMeans(n_clusters=5 , random_state=0)

# Fit the model to the data
kmeans.fit(dt_k)

# Get cluster centers and labels
centers = kmeans.cluster_centers_
labels = kmeans.labels_
dt['cluster'] = labels 
tag =pd.DataFrame(dt['cluster'])
tag = pd.get_dummies(dt['cluster'], prefix='cluster')
tag.replace({False: 0, True: 1})
ch_index = calinski_harabasz_score(dt_k, labels)


C:\Users\S409478\AppData\Local\Temp\ipykernel_15888\1422767163.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tag.replace({False: 0, True: 1})


In [1253]:
#tt2[columns_].columns

In [7]:
ch_index

1902.5803728531569

In [28]:
dt['Tempo abaixo de 10 mbar(min)'].max()

27.167

In [30]:
dt['Tempo abaixo de 10 mbar(min)'].std()

3.4108827724093174

In [8]:
#priorizar o teor de c após a descarburação na otimização.
# optimização:
dt['opt'] = 0
dt.loc[
    (dt['Teor de C após a descarburação(%)'] < 0.009) &
(dt['Temperatura do aço após a descarburação(°C)'] >= 1680) &
(dt['Temperatura do aço após a descarburação(°C)'] <= 1700)
    ,
    'opt'
] = 1


In [10]:
#priorizar o teor de c após a descarburação na otimização.
# optimização:
teste['opt'] = 0
teste.loc[
    (teste['Teor de C após a descarburação(%)'] < 0.009) &
(teste['Temperatura do aço após a descarburação(°C)'] >= 1680) &
(teste['Temperatura do aço após a descarburação(°C)'] <= 1700)
    ,
    'opt'
] = 1


In [11]:
teste['opt'].value_counts()

opt
1    3
0    1
Name: count, dtype: int64

In [12]:
teste['opt']

0    1
1    1
2    0
3    1
Name: opt, dtype: int64

In [52]:
# Primeira Volta com Hubber-Loss
import numpy as np
import pandas as pd
import warnings
from scipy.optimize import minimize

# Custom Huber loss function for logistic regression
def huber_loss(params, X, y, delta=1.0):
    """Calculate Huber loss for logistic regression."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    return np.sum(loss) / n

def huber_loss_grad(params, X, y, delta=1.0):
    """Gradient of Huber loss for logistic regression."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    return grad

# Data Preparation
warnings.filterwarnings('ignore')

columns_ = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]

stp = [[3.521582,	400.0,	9.650573,	4.55,	115.391183]]
contr = pd.DataFrame(stp, columns=columns_)

# Example control DataFrame
ent = teste[['Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)',
       'Teor de Mo antes do VOD(%)', 'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela']]

ent_ = ent.sample(n=1)  # Randomly sample one row for example

# Concatenate and process DataFrames
contr.reset_index(drop=True, inplace=True)
tt2 = pd.concat([ent_, contr], axis=1)
tt2.fillna(0, inplace=True)
tt2 = pd.DataFrame(tt2.sum()).T

# Selecting the features for prediction
tt2_test = tt2[columns_]

# Training Data
x_ = dt[[
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]]

y_ = dt['opt']

# Ensure 'optimal_params' is defined
initial_params = np.zeros(x_.shape[1])
result = minimize(fun=huber_loss, x0=initial_params, args=(x_.values, y_.values),
                  jac=huber_loss_grad, method='BFGS')

optimal_params = result.x

# Predicting on tt2
tt2_features = tt2.values
y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

# Display predictions for tt2
tt2['Predicted_Probability'] = y_pred_tt2_prob
tt2['Predicted_Class'] = y_pred_tt2_binary

print("Predictions on tt2:")
print(tt2[['Predicted_Probability', 'Predicted_Class']], ent_.index)


Predictions on tt2:
   Predicted_Probability  Predicted_Class
0           7.846555e-78                0 Index([1], dtype='int64')


In [53]:
## primaira volta: # inclui a temperatura no final da corrida
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')

columns_ = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
'Pressão média de vácuo durante o sopro de O2(mbar)',
'Tempo abaixo de 10 mbar(min)','Vazão média de argônio durante a descarburação(Nm3/h)','Volume de O2 soprado(m3)'
 ]
   
stp = [[4.9, 483.6, 17.35, 5.67, 208]]
contr = pd.DataFrame(stp, columns = columns_)
# controle df:
ent = teste[['Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)','Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela']]
ent_ = ent.sample(n=1)
contr.reset_index(drop=True, inplace=False)
tt2 = pd.concat([ent_,contr], axis=1)
tt2.fillna(0)
tt2 =pd.DataFrame(tt2.sum())
tt2 = tt2.T
tt2_test = tt2[['Vazão média de argônio durante o sopro de O2(Nm3/h)',
'Pressão média de vácuo durante o sopro de O2(mbar)',
'Tempo abaixo de 10 mbar(min)','Vazão média de argônio durante a descarburação(Nm3/h)','Volume de O2 soprado(m3)'
                
                ]]
x_ = dt[['Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
       'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
       'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
       'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
       'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela', 
        
       'Vazão média de argônio durante o sopro de O2(Nm3/h)',
       'Pressão média de vácuo durante o sopro de O2(mbar)',
      'Tempo abaixo de 10 mbar(min)',
       'Vazão média de argônio durante a descarburação(Nm3/h)',
       'Volume de O2 soprado(m3)']]
y_ = dt['opt']
#tt2_test.columns = x_train_3.columns
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_,y_, test_size = 0.9)
model = LogisticRegression(penalty='l1', solver='saga',class_weight= 'balanced')
model.fit(x_, y_)
y_pred2 = model.predict(tt2)
y_pred_prob2 = model.predict_proba(tt2)
y_pred_score2 = model.score(x_, y_)
###
print(y_pred2 ,y_pred_prob2, ent_.index )

[1] [[0.4846774 0.5153226]] Index([3], dtype='int64')


In [43]:
print(y_pred_tt2_prob,y_pred_tt2_binary)

[7.84655481e-78] [0]


In [22]:
## OFICIAL
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import time
import warnings
import gc

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Precompute common data outside the loop if possible
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        tt2_test = tt2[[
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)'
        ]]

        x_ = dt[[
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)',
            'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
            'Vida de plug da panela', 'Vida da panela',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)'
        ]]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9, random_state=42)

        # Train the Logistic Regression model
        model = LogisticRegression(penalty='l1', solver='saga', class_weight='balanced', C=0.1)
        model.fit(x_, y_)
        y_pred2 = model.predict(tt2)
        y_pred_prob2 = model.predict_proba(tt2)[:, 1]  # Get the probability of the positive class
        y_pred_score2 = model.score(x_test_3, y_test_3)

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train_3, x_train_3)
        results = logit_model.fit(method='bfgs',cov_type = 'HC2', C=0.1)

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in contr.columns:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(contr.columns) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition
        if np.all(y_pred_prob2 >= 0.99) and np.all(y_pred2 == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue

Iteration: 1, Time elapsed: 0.00 seconds
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38
Iteration: 2, Time elapsed: 0.30 seconds
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38
Iteration: 3, Time elapsed: 0.42 seconds
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38
Iteration: 4, Time elapsed: 0.53 seconds
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38
Iteration: 5, Time elapsed: 0.67 seconds
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38
Iteration: 6, Time elapsed: 0.78 seconds
         Current function value: 0.444559
         Iterations: 35
         Function 

In [54]:
## WITH HUBBER_LOSS:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time
import warnings
import gc

# Custom Huber loss function for logistic regression
def huber_loss(params, X, y, delta=1.0):
    """Calculate Huber loss for logistic regression."""
    # Convert inputs to numpy arrays for safety
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    return np.sum(loss) / n

def huber_loss_grad(params, X, y, delta=1.0):
    """Gradient of Huber loss for logistic regression."""
    # Convert inputs to numpy arrays for safety
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 20
iteration = 0

# Define constraints
constraints = {'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)'
        ]

        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9, random_state=42)

        # Optimize using Huber loss
        initial_params = np.zeros(x_train_3.shape[1])
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train_3, x_train_3)
        results = logit_model.fit(method='bfgs', cov_type='HC2')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition
        if np.all(y_pred_tt2_prob >= 0.99) and np.all(y_pred_tt2_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.753187                1
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38
Iteration: 2, Time elapsed: 0.15 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.992754                1
         Current function value: 0.444559
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 38


In [55]:
print(y_pred_tt2_prob, ent_.index)

[0.99275407] Index([3], dtype='int64')


In [56]:
stp_n = tt2[columns_].values

In [57]:
tt2[columns_]

,Vazão média de argônio durante o sopro de O2(Nm3/h),Pressão média de vácuo durante o sopro de O2(mbar),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Volume de O2 soprado(m3)
0,3.521582,483.6,17.35,5.67,208.0


In [58]:
stp

[[4.9, 483.6, 17.35, 5.67, 208]]

In [1581]:
teste.iloc[ent_.index][columns_].values

array([[4.9, 483.6, '17.35', 5.67, 208]], dtype=object)

In [1579]:
stp_n

array([[  3.52158189, 400.        ,   9.65057301,   4.55      ,
        115.39118253]])

In [59]:
impact_var2

,coef,std err,z,P>|z|,[0.025,0.975],exp_coeff
Vazão média de argônio durante o sopro de O2(Nm3/h),-1.8075,0.713,-2.536,0.011,-3.204,-0.411,0.140937


In [ ]:
df_results

In [ ]:
# 4.55 - 7.87

In [ ]:
## Perguntar para o Robinho:
## se o problema á temperatura : "reage mais" e fica , mais tempo abaixo de 10(mbar)
## Tempo abaixo de 10mbr = 26.7, significa quevai precisar de ressopro.
#274

In [ ]:
## Explicação: 
#- Variável mais impactante é o tempo de vácuo, quanto mais melhor
# - Em detrimento dele, o modelo geralmente aumenta a reação, vazão de 02 / vazão de argônio... Ou mantém inalterado.


In [ ]:
## idx[2] = [4.9, 464.3, 17.483, 7, 183]]
#idx